In [1]:
import bagpy
import pandas as pd 
from matplotlib import pyplot as plt 
import numpy as np

In [2]:
from bagpy import bagreader

b = bagreader(r'C:\Users\Teja\Documents\_INFOTECH\Thesis\sample_Ros_bag\2021-03-12_11-41-55_0\2021-03-12_11-41-55_0.bag')



[INFO]  Data folder C:\Users\Teja\Documents\_INFOTECH\Thesis\sample_Ros_bag\2021-03-12_11-41-55_0\2021-03-12_11-41-55_0 already exists. Not creating.


In [3]:
bag = b.reader


topic_list = ['/em/fused_grid', '/sensorik/axis_front/image/compressed', '/vehicle/odometry', '/function/pathing_mlteleop']


pathing_msgs = []
grid_msgs = []
grid_seqs = []
odom_msgs = []
odom_seqs = []
image_msgs = []
image_seqs = []
count = 0
for topic, msg, t in bag.read_messages(topics=topic_list):
    if topic == "/function/pathing_mlteleop":
        pathing_msgs.append(msg)

    if topic == "/em/fused_grid":
        grid_msgs.append(msg)
        grid_seqs.append(msg.header.seq)
        
    if topic == "/vehicle/odometry":
        odom_msgs.append(msg)
        odom_seqs.append(msg.header.seq)
    if topic == "/sensorik/axis_front/image/compressed":
        image_msgs.append(msg)
        image_seqs.append(msg.header.seq)
    if count==10000:
        break
    else: count +=1
bag.close()

print('len')
print('pathing_msgs',len(pathing_msgs))
print('grid_msgs',len(grid_msgs))
print('odom_msgs',len(odom_msgs))
print('image_msgs',len(image_msgs))




len
pathing_msgs 227
grid_msgs 746
odom_msgs 5222
image_msgs 1315


In [12]:

def construct_path(list_of_points):
    path_x = []
    path_y = []
    for path in list_of_points:
        path_x.append(path.point.x)
        path_y.append(path.point.y)
    return path_x,path_y
    

def padd_w_value(list_values,value,max_len):
    list_values = list_values.append([value]*(max_len-len(list_values)))

    return list_values
    

In [13]:
# Example for a single pathing_msg:
init_path_x = []
init_path_y = []
opt_path_x = []
opt_path_y = []

grid_msg = None
image_msg = None
odom_msg = None

max_length = 25
np_init_paths = np.empty((1,max_length,2))

for examplary_pathing_msg in pathing_msgs[0:1]:
    #examplary_pathing_msg = pathing_msgs[0]
    
    print("examplary_pathing_msg")
    #print(examplary_pathing_msg)

    '''
    for init_path in examplary_pathing_msg.path_options[0].reference_path:
        init_path_x.append(init_path.point.x)
        init_path_y.append(init_path.point.y)
    '''
    init_path_x, init_path_y = construct_path(examplary_pathing_msg.path_options[0].reference_path)
    init_path_x = padd_w_value(init_path_x,0,25)
    opt_path_x, opt_path_y = construct_path(examplary_pathing_msg.path_options[2].reference_path)

    '''
    for opt_path in examplary_pathing_msg.path_options[2].reference_path:
        opt_path_x.append(opt_path.point.x)
        opt_path_y.append(opt_path.point.y)
    '''
    # corresponding occupancy grid, camera and odometry sequences (ID's):
    # this saving method is a workaround for the corresponding ID's
    grid_seq = examplary_pathing_msg.path_options[1].reference_path[0].left_boundaries.lane
    image_seq = examplary_pathing_msg.path_options[1].reference_path[0].left_boundaries.road
    odom_seq = examplary_pathing_msg.path_options[1].reference_path[0].left_boundaries.obstacle

    # corresponding occupancy grid, camera and odometry messages:
    """
    grid is saved as vector in grid_msg.data
    grid_msg.info gives information about properties, i.e. width, height, resolution, position in odometry frame
    """
    grid_idx = grid_seqs.index(grid_seq)
    grid_msg = grid_msgs[grid_idx]
    # print('grid_msg.info')
    # print(grid_msg.info)

    """
    1280x720, RGB Image
    """
    image_idx = image_seqs.index(image_seq)
    image_msg = image_msgs[image_idx]
    # Convert compressed image to RAW
    #bridge = CvBridge()
    #img = bridge.compressed_imgmsg_to_cv2(image_msg)
    # print(img.shape)

    """
    odom_msg provides information about the current position in the odometry frame (pos_x, pos_y, heading)
    the idea is to provide the NN information for the correlation between camera image, the corresponding car position and the path to be optimized/predicted
    """
    odom_idx = odom_seqs.index(odom_seq)
    odom_msg = odom_msgs[odom_idx]

examplary_pathing_msg


In [14]:
print((init_path_x))
print(len(init_path_y))

print(len(opt_path_x))
print(opt_path_y)

None
15
15
[-6.57004452795753, -6.378483879163629, -6.180362328252016, -5.973425002693801, -5.762831703165546, -5.548582429667249, -5.327442452104359, -5.102593165955435, -4.874034571220477, -4.637300025642181, -4.395037053777732, -4.147245655627131, -3.8818057128616013, -3.59091470041624, -3.274572618291046]


In [ ]:
class np_maker(pathing_msgs):

    def __init__(self):
        self.pathing_msgs = pathing_msgs
        
    def get_grid_data_numpy():


    def construct_path(list_of_points):
        path_x = []
        path_y = []
        for path in list_of_points:
            path_x.append(path.point.x)
            path_y.append(path.point.y)
        return path_x,path_y
        

    def get_path_data_numpy(self):

        for examplary_pathing_msg in self.pathing_msgs:
            init_path_x, init_path_y = construct_path(examplary_pathing_msg.path_options[0].reference_path)
            opt_path_x, opt_path_y = construct_path(examplary_pathing_msg.path_options[2].reference_path)

        
